In [1]:
import pandas as pd
import numpy as np

# This code displays all results created within a jupyter notebook cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_column', 100)
pd.set_option('display.max_row', 200)

In [2]:
data = pd.read_csv('Data_for_prediction.csv', header=0)

In [3]:
list(data.columns.values)

['Id',
 'neighborhood',
 'larger_neighborhood',
 'Name',
 ' Population',
 'Median Household Income',
 'Median Home Value',
 'Percent Bachelors Degree',
 'Median Rent',
 'Percent Public Transportation to Work',
 'Percent White Non-Hispanic',
 ' Housing Units',
 'Percent Renter Occupied',
 'Percent Occupied Structure w 3-4 Units',
 'Percent Occupied Structure w 5+ Units',
 'Percent Households w 3 Vehicles',
 'Percent Households w 4+ Vehicles',
 'Percent Families, Married w Children Under 18',
 ' Total Households',
 'Year']

In [4]:
Series_tract= []
for j in list(range(480)):
    for i in list(range(8)):
        Series_tract.append(data[' Total Households'].iloc[j+480*i])

In [5]:
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
import numpy

# create a difference transform of the dataset
def difference(dataset):
    diff = list()
    for i in range(1, len(dataset)):
        value = dataset[i] - dataset[i - 1]
        diff.append(value)
    return numpy.array(diff)

# Make a prediction give regression coefficients and lag obs
def predict(coef, history):
        yhat = coef[0]
        for i in range(1, len(coef)):
            yhat += coef[i] * history[-i]
        return yhat

/Users/Angel/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [22]:
# AR
predictions = [1] * 480
for i in list(range(480)):       
    X = difference(Series_tract[8*i:8*i+8])
    history = X[6] 
    model = AR(X)
    model_fit = model.fit(maxlag=1, disp=False)
    window = model_fit.k_ar
    coef = model_fit.params     
    #predictions[i] =  coef[1]+coef[0]*(X[6])
    predictions[i] = model_fit.predict(start=1, end=8, dynamic=False)[0]


In [38]:
# Mean of change percentage
predictions = [1] * 480
for i in list(range(480)):       
    X = np.mean((difference(Series_tract[8*i-8:8*i])/Series_tract[8*i-8]))
    predictions[i] = Series_tract[8*i]*(1+X)*(1+X)*(1+X)

/Users/Angel/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Angel/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [39]:
len(predictions)

480

In [40]:
predictions

[nan,
 1178.2279429920736,
 1095.4149535033407,
 1082.4425210515396,
 1114.5303450334156,
 1021.5255515907469,
 917.2105157647261,
 969.13825549500029,
 1052.9565619921707,
 913.1331690943822,
 953.2973121346987,
 893.83204483016073,
 869.66247017933188,
 915.95155250321295,
 1403.1170316959813,
 940.40108569071185,
 1041.4861883059559,
 880.71115200678491,
 803.50864263365588,
 866.05682133358403,
 970.15152639003543,
 821.52569769530521,
 762.89203228404085,
 773.62495522023983,
 805.5970400869096,
 838.07616719661803,
 856.29506314973025,
 875.56060016215224,
 699.26480514751563,
 812.00734065492531,
 919.05117494582237,
 776.92467152242875,
 732.12567376038896,
 722.93996669729063,
 748.13626088318154,
 788.73200500835537,
 734.84882126146795,
 707.99007492461715,
 693.62130520926462,
 710.11779704982632,
 764.27755560833032,
 755.2555971790473,
 747.90093450675022,
 751.72155232964349,
 938.81246649583909,
 789.79987851801047,
 885.70736071416525,
 871.55799808983261,
 787.2698058

In [172]:
df = pd.DataFrame(predictions)

In [173]:
df.to_csv('prediction_single_var.csv')

### predictions